In [275]:
import requests
import time
import datetime
import pandas as pd
import numpy as np
from pprint import pprint
import json

In [276]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

In [277]:
%pylab inline
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [278]:
import argparse
import ConfigParser
config = ConfigParser.ConfigParser()
config.read('.ttapis.cfg')
config.sections()
fb_config = dict(config.items('Facebook'))

In [280]:
from facebookads.api import FacebookAdsApi
from facebookads import objects

my_app_id = fb_config['ads_app_id']
my_app_secret = fb_config['ads_app_secret']
my_access_token = fb_config['ads_access_token']
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

In [297]:
from facebookads.adobjects.campaign import Campaign

campaign = Campaign(parent_id='act_28013561')

In [298]:
from facebookads.adobjects.campaign import Campaign
from facebookads.adobjects.adsinsights import AdsInsights
from facebookads.adobjects.adreportrun import AdReportRun
from facebookads.adobjects.adcreativelinkdata import AdCreativeLinkData
import time

class CampaignResults():
    def __init__(self):
        self.campaign = Campaign('6050222493443')
        self.params = {
                'level': AdsInsights.Level.ad,
                'breakdowns':  AdsInsights.Breakdowns.placement,
                'fields':[AdsInsights.Field.ad_id,
                        AdsInsights.Field.impressions,
                        AdsInsights.Field.video_10_sec_watched_actions,
                        AdsInsights.Field.unique_actions
             
                         ]
                    }
        pass
    
    def request(self):
        self.insights = self.campaign.get_insights(params=self.params, async=False)
        return self.insights
    
    def build_basic_df(self):
        df = pd.DataFrame(np.nan, index=[0], columns=['empty'])
        records = [dict(insight) for insight in self.insights]
        records = [{metric:record[metric] for metric in record.keys()} for record in records]   
        frames = [pd.DataFrame([pd.Series(record)]) for record in records]
        for frame in frames:
            df = pd.concat([df,frame],axis=0)
        self.df= df.drop(['empty'],axis=1).dropna()
        return self.df
    
    def build_actions(self):
        self.actions_dict = {}
        row_iterator = self.df.iterrows()
        for i, row in row_iterator:
            if row['ad_id'] not in self.actions_dict.keys():
                self.actions_dict[row['ad_id']]={}
        
            self.actions_dict[row['ad_id']][row['placement']]= row['unique_actions']
        return self.actions_dict

    def build_df_actions(self):
        data = self.actions_dict
        frames=[]
        from pandas.io.json import json_normalize
        for ad_id in data:
            for placement in data[ad_id]:
                data[ad_id][placement]={item['action_type']:item['value'] for item in data[ad_id][placement] }
                frame = json_normalize(data[ad_id][placement])
                frame['placement']=placement
                frame['ad_id'] = ad_id
                frames.append(frame)
        df = frames[0]
        for frame in frames:
            df= pd.concat([df,frame], axis=0)
        self.df_actions=df[1:]
        return self.df_actions    

In [299]:
Camres = CampaignResults()
Camres.request()
Camres.build_basic_df()
actions = Camres.build_actions()
Camres.build_df_actions()

,ad_id,comment,like,link_click,page_engagement,photo_view,placement,post,post_engagement,post_like,unlike,video_play,video_view
0,6050294268843,NaN,3.0,797.0,6432,NaN,mobile_feed,21.0,6432,64.0,NaN,359.0,6224.0
0,6050294268843,NaN,NaN,NaN,128,1.0,desktop_video_channel,1.0,128,1.0,NaN,NaN,128.0
0,6050294268843,NaN,NaN,1.0,32,NaN,mobile_video_channel,NaN,32,2.0,NaN,26.0,32.0
0,6050294268843,NaN,NaN,120.0,1712,NaN,mobile_external_only,NaN,1712,NaN,NaN,423.0,1696.0
0,6050294268843,NaN,NaN,16.0,80,36.0,right_hand,NaN,80,NaN,NaN,19.0,64.0
0,6050294268843,44.0,12.0,312.0,39808,148.0,desktop_feed,254.0,39808,559.0,1.0,414.0,39696.0
0,6051482610443,NaN,NaN,12.0,94,NaN,mobile_feed,NaN,94,NaN,NaN,3.0,91.0
0,6051482610443,NaN,NaN,NaN,3,NaN,desktop_video_channel,NaN,3,NaN,NaN,NaN,3.0
0,6051482610443,NaN,NaN,NaN,28,NaN,mobile_video_channel,NaN,28,NaN,NaN,3.0,28.0
0,6051482610443,NaN,NaN,NaN,36,NaN,mobile_external_only,NaN,36,NaN,NaN,2.0,36.0


In [291]:
Camres.build_basic_df()

""
0


In [74]:
from facebookads import FacebookSession
from facebookads import FacebookAdsApi
from facebookads.objects import (
    AdUser,
    AdAccount,
    Campaign,
)

import json
import os
import pprint

pp = pprint.PrettyPrinter(indent=4)
#this_dir = os.path.dirname(__file__)
#config_filename = os.path.join(this_dir, 'config.json')

#config_file = open(config_filename)
#config = json.load(config_file)
#config_file.close()

### Setup session and api objects
session = FacebookSession(
    my_app_id,
    my_app_secret,
    my_access_token,
)
api = FacebookAdsApi(session)

if __name__ == '__main__':
    FacebookAdsApi.set_default_api(api)

    print('\n\n\n********** Reading objects example. **********\n')

    ### Setup user and read the object from the server
    me = AdUser(fbid='me')

    ### Read user permissions
    print('>>> Reading permissions field of user:')
    pp.pprint(me.remote_read(fields=[AdUser.Field.permissions]))

    ### Get first account connected to the user
    my_accounts = list(me.get_ad_accounts())


    ### Read connections (in this case, the accounts connected to me)

    # Pro tip: Use list(me.get_ad_accounts()) to make a list out of
    # all the elements out of the iterator

    my_accounts_iterator = me.get_ad_accounts()
    print('>>> Reading accounts associated with user')
    for account in my_accounts_iterator:
        pp.pprint(account)

        print(">>> Campaign Stats")
        for campaign in account.get_campaigns(fields=[Campaign.Field.name]):
            print campaign
            ##for stat in campaign.get_stats(fields=[
              #  'impressions',
            # 'clicks',
           #     'spent',
           #     'unique_clicks',
        #     'actions',
           # ]):
            print(campaign[campaign.Field.id])
                #for statfield in stat:
                    #print("\t%s:\t\t%s" % (statfield, stat[statfield]))





********** Reading objects example. **********

>>> Reading permissions field of user:
<AdUser> {
    "id": "10100939034793548",
    "permissions": [
        {
            "data": [
                {
                    "permission": "ads_management",
                    "status": "granted"
                },
                {
                    "permission": "ads_read",
                    "status": "granted"
                },
                {
                    "permission": "read_insights",
                    "status": "granted"
                },
                {
                    "permission": "manage_pages",
                    "status": "granted"
                },
                {
                    "permission": "public_profile",
                    "status": "granted"
                }
            ]
        }
    ]
}
>>> Reading accounts associated with user
<AdAccount> {
    "account_id": "10100938920203188",
    "id": "act_10100938920203188"
}
>>> Campaign Stat

In [65]:
my_account.get_campaigns()

[]

In [89]:
from facebookads.objects import Campaign
campaign = Campaign('6050222493443')
params = {
 'date_preset':Campaign.DatePreset.this_week
}
insights = campaign.get_insights(params=params)
print insights

[<AdsInsights> {
    "account_id": "28013561",
    "campaign_id": "6050222493443",
    "date_start": "2016-08-14",
    "date_stop": "2016-08-16",
    "impressions": "147402",
    "spend": 5.05
}]


In [91]:
campaign.get_ads()

[<Ad> {
    "id": "6052443277843"
}, <Ad> {
    "id": "6052360427043"
}, <Ad> {
    "id": "6052224303043"
}, <Ad> {
    "id": "6051482610443"
}, <Ad> {
    "id": "6051259929643"
}, <Ad> {
    "id": "6050874391643"
}, <Ad> {
    "id": "6050645327643"
}, <Ad> {
    "id": "6050644975243"
}, <Ad> {
    "id": "6050294268843"
}, <Ad> {
    "id": "6050223505643"
}, <Ad> {
    "id": "6050222501643"
}]

In [102]:
from facebookads.objects import Ad
ad = Ad('6051482610443')
params = {
 'date_preset':Ad.DatePreset.lifetime
}
insights = ad.get_insights(params=params)
print insights

[<AdsInsights> {
    "account_id": "28013561",
    "ad_id": "6051482610443",
    "adset_id": "6050222494643",
    "campaign_id": "6050222493443",
    "date_start": "2016-07-29",
    "date_stop": "2016-08-16",
    "impressions": "11411",
    "spend": 0.34
}]


In [ ]:
from facebookads.adobjects.campaign import Campaign
from facebookads.adobjects.adsinsights import AdsInsights
from facebookads.adobjects.adreportrun import AdReportRun
from facebookads.adobjects.adcreativelinkdata import AdCreativeLinkData
import time

campaign = Campaign('6050222493443')
params = {
    'level': AdsInsights.Level.ad,
    'breakdowns':  AdsInsights.Breakdowns.placement,
    'fields':[AdsInsights.Field.ad_id,
        AdsInsights.Field.impressions,
               AdsInsights.Field.video_10_sec_watched_actions,
             AdsInsights.Field.unique_actions
             
             ]
}
insights = campaign.get_insights(params=params, async=False)

In [199]:

df = pd.DataFrame(np.nan, index=[0], columns=['empty'])
records = [dict(insight) for insight in insights]
records = [{metric:record[metric] for metric in record.keys()} for record in records]   
frames = [pd.DataFrame([pd.Series(record)]) for record in records]
for frame in frames:
    df = pd.concat([df,frame],axis=0)
df= df.drop(['empty'],axis=1).dropna()


In [249]:
len(df)

44

In [274]:
build_df_actions(df)

KeyError: 'unique_actions'

In [273]:
data = build_df_actions(df)
frames=[]
from pandas.io.json import json_normalize
for ad_id in data:
    for placement in data[ad_id]:
        data[ad_id][placement]={item['action_type']:item['value'] for item in data[ad_id][placement] }
        frame = json_normalize(data[ad_id][placement])
        frame['placement']=placement
        frame['ad_id'] = ad_id
        frames.append(frame)
df = frames[0]
for frame in frames:
    df= pd.concat([df,frame], axis=0)
#df=df[1:]

KeyError: 'unique_actions'

In [270]:
len(df)

45

In [ ]:
async_job.remote_read()

while async_job[AdReportRun.Field.async_percent_completion] < 100:
    time.sleep(1)
    async_job.remote_read()

time.sleep(1)

print(async_job.get_result())

In [118]:
from facebookads.adobjects.ad import Ad
from facebookads.adobjects.adsinsights import AdsInsights

ad = Ad('6052443277843')
insights = ad.get_insights(params={
    'breakdown': AdsInsights.Breakdowns.placement,
})

In [119]:
insights

[<AdsInsights> {
    "account_id": "28013561",
    "ad_id": "6052443277843",
    "adset_id": "6052443274043",
    "campaign_id": "6050222493443",
    "date_start": "2016-07-18",
    "date_stop": "2016-08-16",
    "impressions": "121146",
    "spend": 5.04
}]